<a href="https://colab.research.google.com/github/tae-su-kim/vllm-101/blob/main/samples/colab_vllm_integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# vLLM 실습 101

이 노트북에서는 vLLM library의 셋업 및 사용 방법을 간단히 알아봅니다.

## COLAB runtime 설정 (user action required)

이 노트북은 LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct 모델을 vllm library를 이용해서 실행해보도록 작성되었습니다. 이 노트북은 Google Colab에서 제공하는 GPU 환경을 사용하는 것을 전제로 합니다.

먼저 다음과 같이 Colab Runtime을 설정합니다.

메뉴 -> 런타임 -> 런타임 유형 변경 -> T4 GPU (무료 환경 사용 가능).

# 1. vLLM 라이브러리 설치

In [ ]:
!pip install vllm==0.7.3 requests datasets

# When running vllm directly from source, use this instead
# %load_ext autoreload
# %autoreload 2
# import sys
# import os
# sys.path.append(os.path.abspath('..'))
## os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
## Login to huggingface_hub if we need gated model. Not applicable now.
#from huggingface_hub import notebook_login
#notebook_login()

# 2. LLM 모델 로드

In [ ]:
from vllm import LLM

llm = LLM(model='LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct')

이 셀이 성공적으로 실행되었다면, vLLM 설정과 huggingface 접근이 잘 수행된 것입니다.

추가로 현재 노트북에서 display를 개선하기 위한 코드를 실행합니다 (vLLM과 관련 없음).

In [ ]:
from IPython.display import display, Markdown

def display_header(text):
    display(Markdown(f'**{text}**'))

def display_content(text):
    display(Markdown(f'```\n{text}\n```'))

## 해당 언어 모델을 위한 시스템 프롬프트 준비
이 시스템 프롬프트는 [LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct](https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct) 모델의 repository에서 예시로 사용하고 있는 프롬프트입니다. 프롬프트는 활용에 따라서 자유롭게 수정할 수 있고, 많은 경우에 실제 유저 경험에도 큰 영향을 미치게 됩니다. 자유롭게 변경해보세요!

In [ ]:
SYSTEM_PROMPT = """\
You are EXAONE model from LG AI Research, a helpful assistant.\
"""

def get_prompt(message: str, system_prompt: str = SYSTEM_PROMPT) -> str:
    return f'<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{message} [/INST]'

## Offline inference 테스트

위에서 생성한 vllm.LLM 인스턴스를 통해 LLM 모델의 offline inference를 실험해볼 수 있습니다. Offline inference는 API 서빙을 위한 batching 등 front-end 기능을 제외한 것으로, LLM inference 자체를 실험 해 보고 싶을때, 혹은 로컬 스크립트에서 LLM 기능이 필요할때 사용할 수 있습니다.

[SamplingParams](https://docs.vllm.ai/en/v0.6.0/dev/sampling_params.html)는 llm 모델의 출력 토큰 생성시에 sampling 과정을 정의할 수 있는 변수로, top_p, top_k, temperature 등 여러 파라미터를 수정하여 output을 바꿀 수 있습니다. 해당 옵션들은 서빙 성능에도 영향이 있고, 모델 성능에도 영향이 있으므로 활용처에 따라 다르게 세팅할 필요가 있습니다.

In [ ]:
from vllm import SamplingParams

sampling_params = SamplingParams()
sampling_params.max_tokens = 100
sampling_params.top_p = 0.5
sampling_params.top_k = 10
sampling_params.temperature = 0.7

question = "판교역에 대해 알려줘."
prompt = get_prompt(question)

display_header("Prompt:")
display_content(prompt)

display_header("Answer:")
result = llm.generate(prompt, sampling_params=sampling_params)
display_content(result[0].outputs[0].text)


결과가 성공적으로 출력되었다면, vLLM의 기본적인 사용에 성공하셨습니다!

## Online Serving 테스트

Offline Serving에 성공하였으니, 이제 Online Serving API를 띄워봅니다. LLM의 Online Serving은 보통 OpenAI API-compatible한 포맷으로 지원하는 경우가 많으며, vllm에서도 이를 지원하고 있습니다.

In [37]:
import subprocess
import time
import os

# del LLM

# Start vllm server in the background
vllm_process = subprocess.Popen([
    'vllm',
    'serve',  # Subcommand must follow vllm
    'LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct',
    '--trust-remote-code',
    '--dtype', 'half',
    '--max-model-len', '2048',
], stdout=subprocess.PIPE, stderr=subprocess.PIPE, start_new_session=True)

## vLLM API 상태 확인하기

서빙 API를 subprocess로 실행하였으므로 해당 프로세스에 직접 HTTP 요청을 통해 상태를 확인합니다.

In [ ]:
import requests

def check_vllm_status():
    try:
        response = requests.get("http://localhost:8000/health")
        if response.status_code == 200:
            print("vllm server is running")
            return True
    except requests.exceptions.ConnectionError:
        print("vllm server is not running")
        return False

try:
    # Monitor the process
    while True:
        if check_vllm_status() == True:
            print("The vllm server is ready to serve.")
            break
        else:
            print("The vllm server is not responding.")
        time.sleep(5)  # Check every second
except KeyboardInterrupt:
    print("Stopping the check of vllm...")

## 서빙 성능 벤치마크 해보기

In [ ]:
!git clone https://github.com/vllm-project/vllm
!mkdir /content/data && cd /content/data && wget https://huggingface.co/datasets/anon8231489123/ShareGPT_Vicuna_unfiltered/resolve/main/ShareGPT_V3_unfiltered_cleaned_split.json

In [ ]:
!cd /content/vllm && python benchmarks/benchmark_serving.py \
                        --backend vllm \
                        --model LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct \
                        --dataset-name random \
                        --random-input-len 256 \
                        --random-output-len 256 \
                        --num-prompts 100

2025-03-25 03:28:42.552383: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742873322.809273   12677 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742873322.878753   12677 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-25 03:28:43.416775: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO 03-25 03:29:03 __init__.py:207] Automatically detected platform cuda.
Namespace(backend='vllm', base_url=None, h

## 프로파일링 해보기

torch profiler를 활용하여 vLLM을 프로파일링 해볼 수 있습니다. 아래의 코드를 실행하여 profiler를 킨 채로 offline inference를 진행합니다.

In [ ]:
import os
import time

# delete previous LLM instance
try:
    del llm
except:
    pass

from vllm import LLM, SamplingParams

os.environ["VLLM_TORCH_PROFILER_DIR"] = "./vllm_profile"
llm = LLM(model='LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct')

sampling_params = SamplingParams()
sampling_params.max_tokens = 10
sampling_params.top_p = 0.5
sampling_params.top_k = 10
sampling_params.temperature = 0.7

llm.start_profile()
_ = llm.generate("판교역에 대해 알려줘.", sampling_params=sampling_params)
llm.stop_profile()
time.sleep(10)

위의 코드 실행 이후, 파일 탭에서 vllm_profile 디렉토리 내에 프로파일링 결과 파일이 생성된 것을 확인할 수 있습니다.
파일 탭에서 우클릭하여 해당 파일을 다운로드 할 수 있으며, 다운로드 이후 [perfetto](https://ui.perfetto.dev/)를 이용해 열어볼 수 있습니다.